# Explaining text classifications 


See [Transformers Interpret Multiclass Classification Example](https://github.com/cdpierse/transformers-interpret/blob/master/notebooks/multiclass_classification_example.ipynb)

In [1]:
# Adds multiclass_classification.pkz to sys.path which includes a captum entry_point for explaining integrated gradients. See multclass_classification.yaml
# To create the multiclass_classification.pkz run 'explainer export multiclass_classification'

from explainer.explainers import multiclass_classification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Import Industry Classification Mode
This finetuned model by @sampathkethineedi uses a distilbert base to predict the professional industry a text is referring to. 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

Downloading:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Let's explore the classes, there are 62 unique classes. Many of these are overlapping/related industries such as __Health Care Equipment__ and __Health Care Supplies__.

In [3]:
model.config.id2label

{0: 'Advertising',
 1: 'Aerospace & Defense',
 2: 'Apparel Retail',
 3: 'Apparel, Accessories & Luxury Goods',
 4: 'Application Software',
 5: 'Asset Management & Custody Banks',
 6: 'Auto Parts & Equipment',
 7: 'Biotechnology',
 8: 'Building Products',
 9: 'Casinos & Gaming',
 10: 'Commodity Chemicals',
 11: 'Communications Equipment',
 12: 'Construction & Engineering',
 13: 'Construction Machinery & Heavy Trucks',
 14: 'Consumer Finance',
 15: 'Data Processing & Outsourced Services',
 16: 'Diversified Metals & Mining',
 17: 'Diversified Support Services',
 18: 'Electric Utilities',
 19: 'Electrical Components & Equipment',
 20: 'Electronic Equipment & Instruments',
 21: 'Environmental & Facilities Services',
 22: 'Gold',
 23: 'Health Care Equipment',
 24: 'Health Care Facilities',
 25: 'Health Care Services',
 26: 'Health Care Supplies',
 27: 'Health Care Technology',
 28: 'Homebuilding',
 29: 'Hotels, Resorts & Cruise Lines',
 30: 'Human Resource & Employment Services',
 31: 'IT Co

Import __SequenceClassificationExplainer__ from transformers interpret. This class should work with most if not all language models with a sequence classification head from the transformers package. 


In [4]:
sample_text = """
Stocks ended a choppy session mixed as investors digested a host of corporate earnings results and considered policymakers’ next moves to support the still virus-stricken economy.
The S&P 500 shook off earlier declines to narrowly eke out a record closing high.The Dow ended a tick below its recent record closing level."""

In [5]:
import entry_point
multiclass_explainer = entry_point.entry_point(model, tokenizer)

In [6]:
# call the exlplainer
word_attributions = multiclass_explainer(text=sample_text)

In [7]:
# seems to be an appropriate prediction
multiclass_explainer.predicted_class_name

'Investment Banking & Brokerage'

In [8]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', 0.17552726310233607),
 ('ended', -0.038503090711732704),
 ('a', 0.005213849882699535),
 ('chop', -0.3711210736862698),
 ('##py', 0.0547408919244658),
 ('session', 0.11222962157097703),
 ('mixed', 0.04523504182457808),
 ('as', 0.008543752839572896),
 ('investors', 0.22552279940165373),
 ('digest', 0.09217338299811242),
 ('##ed', 0.1464443040425804),
 ('a', -0.10249613330509676),
 ('host', -0.07218371648238972),
 ('of', 0.22421010032188932),
 ('corporate', 0.2789596257839014),
 ('earnings', 0.6027407244671367),
 ('results', -0.0996496017223695),
 ('and', -0.01712193120579429),
 ('considered', 0.24777286314104055),
 ('policy', 0.1037566231158218),
 ('##makers', 0.011601823072946571),
 ('’', 0.01130267825568464),
 ('next', -0.046363195294348646),
 ('moves', -0.004164080534027568),
 ('to', 0.08991352552802684),
 ('support', 0.07168823763478642),
 ('the', 0.027829731902619767),
 ('still', 0.07106038609355107),
 ('virus', 0.1065563701692622),
 ('-', -0.07300296139

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to this prediction. **Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

In [9]:
html = multiclass_explainer.visualize()

## Explaining The Same Text For A Different Class 
Lets say we think this text could also fall somewhat under the class of __Asset Management & Custody Banks__ If we want it is also possible to get an explantion/attributions for the text with that class

In [10]:
word_attributions = multiclass_explainer(sample_text, class_name="Asset Management & Custody Banks")

In [11]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.14385777183491444),
 ('ended', 0.17111795657175347),
 ('a', -0.06059827675821222),
 ('chop', 0.5140052958408691),
 ('##py', -0.08750464917399801),
 ('session', -0.07754135127683912),
 ('mixed', 0.036185013338416054),
 ('as', -0.009426752579071787),
 ('investors', 0.49884473676649377),
 ('digest', -0.21302128337432716),
 ('##ed', -0.06429115723373122),
 ('a', 0.1777262852180482),
 ('host', 0.15808861100676133),
 ('of', -0.06705092596928693),
 ('corporate', -0.1500207797714331),
 ('earnings', -0.23341496761815866),
 ('results', 0.12625867570460297),
 ('and', 0.05719577984387228),
 ('considered', -0.11990910725910534),
 ('policy', 0.14461768427548477),
 ('##makers', 0.15514263156393673),
 ('’', -0.02760065974180801),
 ('next', 0.02122594440663125),
 ('moves', 0.10101869817226436),
 ('to', -0.05263526480133935),
 ('support', -0.0014980582591976766),
 ('the', 0.08523796705426279),
 ('still', 0.003591138970311294),
 ('virus', 0.008249709450432019),
 ('-', -0.0

The results are close to the first visualization, a good sign that the model is generalizing well for both of these related classes

In [12]:
html = multiclass_explainer.visualize()

What if we get attributions for class that makes no sense in this context such as __Restaurants__?


In [13]:
word_attributions = multiclass_explainer(sample_text, class_name="Restaurants")

There isn't much to this prediction, it is worth nothing however that the words "choppy" had a more positive impact in this instance which seems plausible given the industry. 

In [14]:
html = multiclass_explainer.visualize()

In [15]:
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.6854856776699227),
 ('ended', -0.029031903529460487),
 ('a', 0.11090255712178287),
 ('chop', 0.006652041990268798),
 ('##py', -0.2062490196552602),
 ('session', -0.07065543435255575),
 ('mixed', -0.13288207672076993),
 ('as', 0.07492878699001666),
 ('investors', -0.12119271331548742),
 ('digest', -0.1244596117883646),
 ('##ed', -0.09281783829031225),
 ('a', 0.04926226759885049),
 ('host', 0.0024580264792544724),
 ('of', -0.04181013614866942),
 ('corporate', 0.09596872076394984),
 ('earnings', 0.1978076717838316),
 ('results', 0.2649823840552865),
 ('and', 0.10765583459325372),
 ('considered', 0.08802602452816315),
 ('policy', 0.11527477012595103),
 ('##makers', -0.09797674433730999),
 ('’', -0.04952405611933375),
 ('next', -0.08798394731928101),
 ('moves', 0.016330798640794356),
 ('to', -0.013984953425966241),
 ('support', -0.07466644958978419),
 ('the', 0.018439054280971545),
 ('still', -0.07228693535812657),
 ('virus', 0.07444844928005309),
 ('-', -0.0